# Train ML

> A collection of machine learning tools

In [ ]:
#| default_exp train

In [ ]:
#| hide
import sys
sys.path.append("/notebooks/katlas")
from nbdev.showdoc import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
#| output: False
from sklearn import set_config
from katlas.core import Data
from katlas.feature import *
from fastbook import *
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.stats import spearmanr,pearsonr
from sklearn.model_selection import *
from pathlib import Path
from sklearn.metrics import mean_squared_error
import math
from scipy.stats import spearmanr, pearsonr
from joblib import dump, load
from sklearn.linear_model import *
from sklearn.svm import *
from sklearn.ensemble import *


In [ ]:
#| export
from sklearn import set_config
set_config(transform_output="pandas")

## Splitter

In [ ]:
#| export
def get_splits(df, # df contains info for split
               stratified=None, # colname to make stratified kfold; sampling from different groups
               group=None, # colname to make group kfold; test and train are from different groups
               nfold=5,
              seed=123):
    # train_idx, test_idx = None, None
    
    splits = []
    if stratified is not None and group is None:
        kf = StratifiedKFold(nfold, shuffle=True, random_state=seed)
        for split in kf.split(df.index, df[stratified]):
            splits.append(split)
        print(kf)
        split = splits[0]
        print(f'# kinase {stratified} in train set: {df.loc[split[0]][stratified].unique().shape[0]}')
        print(f'# kinase {stratified} in test set: {df.loc[split[1]][stratified].unique().shape[0]}')
        
    elif group is not None and stratified is None:
        kf = GroupKFold(nfold)
        for split in kf.split(df.index, groups=df[group]):
            splits.append(split)
            
        print(kf)
        split = splits[0]
        print(f'# kinase {group} in train set: {df.loc[split[0]][group].unique().shape[0]}')
        print(f'# kinase {group} in test set: {df.loc[split[1]][group].unique().shape[0]}')
        
    elif stratified is not None and group is not None:
        kf = StratifiedGroupKFold(nfold, shuffle=True, random_state=seed)
        for split in kf.split(df.index, groups=df[group], y=df[stratified]):
            splits.append(split)
            
        print(kf)    
        split = splits[0]
        print(f'# kinase {stratified} in train set: {df.loc[split[0]][stratified].unique().shape[0]}')
        print(f'# kinase {stratified} in test set: {df.loc[split[1]][stratified].unique().shape[0]}')
    else:
        raise ValueError("Either 'stratified' or 'group' argument must be provided.")
        
        
    print('---------------------------')
    print(f'# kinase in train set: {df.loc[split[0]].kinase.unique().shape[0]}')
    
    print('---------------------------')
    print(f'# kinase in test set: {df.loc[split[1]].kinase.unique().shape[0]}')
    print('---------------------------')
    print(f'test set: {df.loc[split[1]].kinase.unique()}')
    
    return splits

In [ ]:
#| export
def split_data(df, # dataframe of values
               feat_col, # feature columns
               target_col, # target columns
               split # one of the split in splits
              ):
    
    X_train = df.loc[split[0]][feat_col]
    y_train = df.loc[split[0]][target_col]
    
    X_test = df.loc[split[1]][feat_col]
    y_test = df.loc[split[1]][target_col]
    
    return X_train, y_train, X_test, y_test

## ML Trainer

In [ ]:
#| export
def train_ml(df, # dataframe of values
             feat_col, # feature columns
             target_col, # target columns
             split, # one split in splits
             model,  # a sklearn models
             save = None, # file (.joblib) to save, e.g. 'model.joblib'
             params={},
            ):
    
    " Train one split of data. Need to specify dataframe, feature columns, target columns, split, and which sklearn models to use"
    
    # split data
    X_train, y_train, X_test, y_test = split_data(df, feat_col, target_col, split)
    
    # Fit the model
    model.fit(X_train, y_train, **params) # better convert y_train to numpy array and flatten
    print(model)
    
    if save is not None:
        # Save the model to a file
        # joblib.dump(model, save)
        dump(model, save)
        
    # Predict train
    y_train_pred = model.predict(X_train) # X_test is dataframe, y_pred is numpy array
    print(f'training set mse: {mean_squared_error(y_train, y_train_pred)}')
    
    # Predict test
    y_pred = model.predict(X_test) # X_test is dataframe, y_pred is numpy array
    print(f'test set mse: {mean_squared_error(y_test, y_pred)}')
    y_pred = pd.DataFrame(y_pred,index=y_test.index, columns = y_test.columns)
    
    return y_test, y_pred #two dataframes

In [ ]:
#| export
def predict_ml(df, # Dataframe that contains features
               feat_col, # feature columns
               model_pth # models.joblib
              ):
    
    test = df[feat_col]
    
    model = load(model_pth)
    
    pred = model.predict(test)
    
    pred_df = pd.DataFrame(pred)
    
    return pred_df

In [ ]:
#| export
def score_all(target, pred):
    
    "Calculate the overall correlation between two dataframes; need to have same index and columns"
    
    # Calculate RMSE
    mse = mean_squared_error(target, pred)
    # rmse = math.sqrt(mse)
    print(f'mse is {mse:.4f}')

    # Calculate the Spearman correlation coefficient
    spearman_corr, _ = spearmanr(target.values.ravel(), pred.values.ravel())
    print(f"Spearman correlation coefficient: {spearman_corr:.4f}")

    # Calculate the Pearson correlation coefficient
    pearson_corr, _ = pearsonr(target.values.ravel(), pred.values.ravel())
    print(f"Pearson correlation coefficient: {pearson_corr:.4f} ")
    
    # return mse,spearman_corr, pearson_corr

### Cross-Validation

In [ ]:
#| export
def train_cv(df, # dataframe of values
             feat_col, # feature columns
             target_col,  # target columns
             splits, # splits
             model, # sklearn model
             save_name = None, # model name to be saved, e.g., 'LR'
             params = {}
            ):
    OOF = []
    for fold, split in enumerate(splits):
        print(f'------ fold: {fold} --------')
        
        if save_name is not None: 
            target, pred = train_ml(df, feat_col, target_col, split, model, f'models/{save_name}_{fold}.joblib',params)
        else:
            target, pred = train_ml(df, feat_col, target_col, split, model, params=params)

        score_all(target,pred)
        OOF.append(pred)
        
    oof_df = pd.concat(OOF).sort_index()
    
    return oof_df

### Score

In [ ]:
#| export
def score_each(target, 
               pred,
               absolute=False, # If absolute, then will get absolute value of spearman and pearson
              ):
    "Calculate spearman and pearson per row"
    
    # pred.columns = target.columns
    sp = target.corrwith(pred,axis=1,method='spearman')
    
    pear = target.corrwith(pred,axis=1,method='pearson')
    
    df = pd.DataFrame(np.stack([sp,pear]).T,columns = ['spearman','pearson'])
    if absolute ==True:
        df = df.apply(abs)
        
    print(f'average spearman for each row is {df.spearman.mean()}')
    print(f'average pearson for each row is {df.pearson.mean()}')
    return df

## XGB

In [ ]:
#| export
def xgb_trainer(df,
                feature_col,
                target_col,
                test_index=None,
                xgb_params = { 
                            'max_depth':7, #from 4 to 7
                            'learning_rate':0.001, #from 0.001
                            'subsample':0.8,
                            'colsample_bytree':1, # from 0.2 to 1, because need to take all features
                            'eval_metric':'rmse',
                            'objective':'reg:squarederror',
                            'tree_method':'gpu_hist',
                            'predictor':'gpu_predictor',
                            'random_state':123
                        },
                model_file='xgb_model.bin',
                split_seed = 123, # seed of random split
               ):
    
    X = df[feature_col]
    y = df[target_col]
    
    print(f'xgb params is: {xgb_params}')
    
    if test_index is None:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=split_seed)
    else:
        X_train,y_train = X.loc[~X.index.isin(test_index)],y.loc[~X.index.isin(test_index)]
        X_test, y_test = X.loc[test_index],y.loc[test_index]

        
    print(X_train.shape,y_train.shape,X_test.shape, y_test.shape)
    print(y_test.index)
    #prepare matrix for xgb
    dtrain = xgb.DMatrix(X_train, y_train)
    dtest = xgb.DMatrix(X_test, y_test)
    
    model = xgb.train(xgb_params, 
            dtrain=dtrain,
            evals=[(dtrain,'train'),(dtest,'valid')],
            num_boost_round=9999,
            early_stopping_rounds=100,
            verbose_eval=100,)
    
    # Save the model
    path = Path(model_file)
    
    # Make a directory if not exists
    path.parent.mkdir(exist_ok=True)
        
    model.save_model(model_file)
    print(f'Model saved to {model_file}')
    
    # Prepare the pred/target df
    pred = model.predict(dtest)
    
    out = np.vstack([np.ravel(y_test),np.ravel(pred)]).T
    pred_df = pd.DataFrame(out,index=y_test.index, columns = ['target','pred'] )
    
    spearman_corr, _ = spearmanr(pred_df.target, pred_df.pred)
    print(f'Spearman correlation: {spearman_corr:.2f}')
    pearson_corr, p_value = pearsonr(pred_df.target, pred_df.pred)
    print(f'Pearson correlation: {pearson_corr:.2f}')


    
    fig, ax = plt.subplots()
    ax.scatter(pred_df.target, pred_df.pred)
    ax.set_xlabel('True values')
    ax.set_ylabel('Predicted values')
    ax.set_title('Scatter plot of true versus predicted values')
    plt.show()
    plt.close()
    
    
    dd = model.get_score(importance_type='gain')
    gain = pd.DataFrame({'feature':dd.keys(),f'gain_importance':dd.values()}).set_index('feature').sort_values(by='gain_importance',ascending=False)
    gain[:10].plot.barh()
    plt.show()
    plt.close()
        
    dd = model.get_score(importance_type='weight')
    weight = pd.DataFrame({'feature':dd.keys(),f'weight_importance':dd.values()}).set_index('feature').sort_values(by='weight_importance',ascending=False)
    weight[:10].plot.barh()
    plt.show()
    plt.close()
    
    return pred_df, gain, weight

In [ ]:
#| export
def xgb_predict(df, # a dataframe that contains ID and features for prediction
                feature_col, #feature column name
                ID_col = "ID", #ID column name
                model_file='xgb_model.bin'):
    # Load the XGBoost model
    model = xgb.Booster()
    model.load_model(model_file)
    
    # Prepare data for prediction
    X = df[feature_col]
    dtest = xgb.DMatrix(X)
    
    # Make predictions
    preds = model.predict(dtest)
    
    # Combine predictions with IDs into a DataFrame
    result_df = pd.DataFrame({ID_col: df[ID_col], 'preds': preds})
    
    return result_df

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()